## Description
__________

This script reads the .nc forcing files generated by 01_generate_ensemble.ipynb and runs the model for each  
year and ensemble member found in the files. The script will automatically run the model for every year of  
forcings found in the esp_forcing_directory. The outputs are saved as .nc files containing basin output  
variables for each year and ensemble member. 

### Requirements
_______

- Set resultsdir in info.txt to ./ (current directory)
  - For example: 
    resultdir ./
- Define a basin output file for the subbasin to be analyzed containing variables cout, rout and snow  
  representing simulated flow, observed flow and snow water equivalent
  - For example: 
    basinoutput variable cout rout snow
    basinoutput meanperiod 1
    basinoutput decimals 3
    basinoutput subbasin 58213
    
Note: This script will automatically change the bdate and edate (start and end date)  of the simulation to  
match what is defined in the forcing files. 

### Import Libraries

In [1]:
import os
import pandas as pd
from datetime import datetime
import xarray as xr
import numpy as np
import shutil
import subprocess
import time

### Inputs

In [2]:
# esp ensemble decisions
subbasin_id= '58232'     # the name of the subbasin to analyze
hype_command= r'.\HYPE_nobutton_DA.exe'     # default is .\HYPE_nobutton_DA.exe for windows

esp_forcing_directory= '../project/esp_forcings/' # directory containing esp forcings (default is esp_forcings)
model_directory= '../project/hype_model/'    # directory containing model files (default is model)
output_directory= '../'         # desired location of output file

### Run Ensemble

In [3]:
# Record start time
start_time = time.time()

In [4]:
# Save current directory
current_directory = os.getcwd()

In [5]:
# Define the working directory path, default is the location of this script
working_directory = "esp_working_directory"

In [6]:
# Create the output directory name
output_file = f"{subbasin_id}_esp_results"

# Create the full output directory path
output_directory = os.path.join(output_directory, output_file)

# Create the output directory if it doesn't already exist
os.makedirs(output_directory, exist_ok=True)

In [7]:
# get list of directories containing forcings for each esp year excluding files starting with .
esp_directories = [name for name in os.listdir(esp_forcing_directory) if os.path.isdir(os.path.join(esp_forcing_directory, name)) and not name.startswith('.')]

In [8]:
# Copy model files into working directory
model_contents = os.listdir(model_directory)
for item in model_contents:
    source = os.path.join(model_directory, item)
    destination = os.path.join(working_directory, item)
    if not os.path.exists(destination):
        if os.path.isdir(source):
            shutil.copytree(source, destination)
        else:
            shutil.copy2(source, destination)

In [9]:
# Iterate over each year of esp 
for forcing_directory in esp_directories:
    
    # Get list of each forcing .nc file in the directory
    file_names = os.listdir(os.path.join(esp_forcing_directory, forcing_directory))

    # Filter out files that aren't .nc
    file_names = [file for file in file_names if file.endswith('.nc')]

    # Initialize list of forcing files
    forcings = {}
    
    # For each .nc file in forcing files
    for file_name in file_names:
        try:
            # Read the forcing file with xarray
            dataset = xr.open_dataset(os.path.join(esp_forcing_directory, forcing_directory, file_name))

            # Name the dataset after the correspoinding HYPE input file without the .nc extension
            dataset_name = os.path.splitext(file_name)[0]
            forcings[dataset_name] = dataset

        except Exception as e:
            print(f"Error reading file {file_name}: {e}")
            
    # extract the analysis year by looking at the Pobs forcing file (this file is mandatory for model runs)
    esp_date= forcings['Pobs'].time.values[-1]
    esp_date_str = str(esp_date)
    esp_year = esp_date_str.split("-")[0] # extract year 
    
    # extract first and last date
    new_bdate_value = forcings['Pobs'].time.values[0]
    new_edate_value = forcings['Pobs'].time.values[-1]

    # find the number of ensemble members from the forcing files
    num_ensemble_members= dataset.sizes['ensemble_member']
    
    # Uncomment this line for debugging, will only go through 3 members
    # num_ensemble_members= 3
    
    # Read the info file and overwrite time series to match the forcing dates
    with open(f'./{working_directory}/info.txt', 'r+') as f:
        lines = f.readlines()
        f.seek(0)
        for line in lines:
            if line.startswith('bdate'):
                f.write(f'bdate {new_bdate_value}\n')
            elif line.startswith('edate'):
                f.write(f'edate {new_edate_value}\n')
            elif line.startswith('cdate'):
                f.write(f'cdate {new_bdate_value}\n')
            else:
                f.write(line)
        f.truncate()


    # Create an empty list to store the results for each ensemble member
    ensemble_member_list = []
    
    # Loop through each ensemble member
    for ensemble_member_num in range(0, num_ensemble_members):

        # Iterate through each forcing file 
        for dataset_name, dataset in forcings.items():
            try:

                # Extract data for given ensemble member
                data = dataset.sel(ensemble_member=ensemble_member_num)

                # Convert data to DataFrame
                data = data.to_dataframe()

                # Modify format to match HYPE input requirements
                # Drop 'ensemble_member' column 
                data = data.drop('ensemble_member', axis=1)

                # Pivot table to have time in index and subbasin as headers
                data = data.pivot_table(index='time', columns='subbasin')

                # Remove the word "subbasin" from the header label
                data.columns = pd.MultiIndex.from_tuples([(col[0], col[1]) if col[0] != 'subbasin' else (None, col[1]) for col in data.columns])

                # Save DataFrame to tab-separated text file
                output_file = dataset_name.replace('_ensemble_1', '') + '.txt'
                data.to_csv(f'{working_directory}/{output_file}', sep='\t', header=True, index=True, index_label=False)

                # Remove the extra line above the header
                with open(f'{working_directory}/{output_file}', 'r') as f:
                    lines = f.readlines()

                with open(f'{working_directory}/{output_file}', 'w') as f:
                    f.writelines(lines[1:])

                # Add index label to files
                with open(f'{working_directory}/{output_file}', 'r') as file:
                    # Read all lines from the file
                    lines = file.readlines()

                lines[0] = 'date ' + lines[0]

                with open(f'{working_directory}/{output_file}', 'w') as file:
                    file.writelines(lines)    

            except Exception as e:
                print(f"Error processing dataset {dataset_name}: {e}")

        # Run HYPE for the data in the given ensemble member 
        result = subprocess.run(hype_command, shell=True, cwd=working_directory)

        # Check the return code
        if result.returncode == 0:
            print("Command executed successfully.")
        else:
            print("Error executing command. Return code:", result.returncode)

        # Read the basin output file for the specified subbasin
        results = pd.read_csv(f'{working_directory}/00{subbasin_id}.txt', sep='\t', index_col=0)  # Change delimiter if needed
        results = results.iloc[1:]

        # Convert the results to an xarray item
        ds = results.to_xarray()

        # Add the current ensemble member as a coordinate
        ds.coords["ensemble_member"] = ensemble_member_num

        # Append the DataArray to the list
        ensemble_member_list.append(ds)
        
        print(f'Ensemble member {ensemble_member_num} complete')
    

    # Concatenate ensemble_member DataArrays along the ensemble_member dimension for the year of analysis
    final_ds = xr.concat(ensemble_member_list, dim='ensemble_member')
    
    
    # Save the concatenated dataset to the specified file path
    final_ds.to_netcdf(f'{output_directory}/{esp_year}_esp.nc')
    
    print(f'Year {esp_year} Completed')

Command executed successfully.
Ensemble member 0 complete
Command executed successfully.
Ensemble member 1 complete
Command executed successfully.
Ensemble member 2 complete
Command executed successfully.
Ensemble member 3 complete
Command executed successfully.
Ensemble member 4 complete
Command executed successfully.
Ensemble member 5 complete
Command executed successfully.
Ensemble member 6 complete
Command executed successfully.
Ensemble member 7 complete
Command executed successfully.
Ensemble member 8 complete
Command executed successfully.
Ensemble member 9 complete
Command executed successfully.
Ensemble member 10 complete
Command executed successfully.
Ensemble member 11 complete
Command executed successfully.
Ensemble member 12 complete
Command executed successfully.
Ensemble member 13 complete
Command executed successfully.
Ensemble member 14 complete
Command executed successfully.
Ensemble member 15 complete
Command executed successfully.
Ensemble member 16 complete
Command

In [10]:
# Delete the working directory and its contents
shutil.rmtree(working_directory)

In [11]:
# Record end time
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.2f} seconds")

Execution time: 15491.92 seconds
